In [7]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import VOCDetection
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Define the transformation for training data
def get_transform(train):
    transforms = [T.ToTensor()]
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# Function to recursively move targets to device
def move_to_device(target, device):
    if isinstance(target, dict):
        return {k: move_to_device(v, device) for k, v in target.items() if isinstance(v, (list, torch.Tensor))}
    elif isinstance(target, list):
        return torch.tensor(target).to(device)
    elif isinstance(target, torch.Tensor):
        return target.to(device)
    else:
        return target

# Load your custom dataset (here using VOC as an example)
dataset = VOCDetection('/path/to/VOC', year='2012', image_set='train', download=True, transform=get_transform(train=True))
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

# Load a pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one (here assuming 3 classes: background, class1, class2)
num_classes = 3
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)

        # Move targets to the device recursively
        targets = [move_to_device(t, device) for t in targets]

        # Debugging: print target structure
        print("Target structure:", targets)

        try:
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
        except KeyError as e:
            print(f"Missing key in target: {e}")
            print("Target structure:", targets)
            continue

    print(f"Epoch {epoch} completed.")

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_model.pth')


Streaming output truncated to the last 5000 lines.
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structure: [{}, {}]
Target structure: [{}, {}]
Missing key in target: 'boxes'
Target structu

In [9]:
!pip install streamlit
import streamlit as st
from PIL import Image
import torch
from torchvision import models, transforms

# Load your fine-tuned model
num_classes = 3  # Replace with the actual number of classes
in_features = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).roi_heads.box_predictor.cls_score.in_features
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load('fine_tuned_model.pth'))
model.eval()

# Define your custom classes
CUSTOM_CLASSES = ['__background__', 'class1', 'class2']  # Replace with actual class names

# Define the Streamlit app
def main():
    st.title("Image Component Detection")
    st.write("Upload an image and click 'Analyse Image' to detect objects in the image.")

    # Image upload widget
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption='Uploaded Image.', use_column_width=True)

        if st.button('Analyse Image'):
            # Image transformation
            transform = transforms.Compose([
                transforms.ToTensor()
            ])
            image_tensor = transform(image).unsqueeze(0)

            # Perform object detection
            with torch.no_grad():
                outputs = model(image_tensor)

            # Extract detected objects and their labels
            detected_objects = outputs[0]['labels'].numpy()
            detected_names = [CUSTOM_CLASSES[i] for i in detected_objects]

            st.write("Detected Components:")
            for name in detected_names:
                st.write(name)

if __name__ == '__main__':
    main()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]
2024-06-21 14:50:25.127 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [1]:
!npm install localtunnel
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 4.543s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details
Password/Enpoint IP for localtunnel is: 34.106.92.60
npx: installed 22 in 5.963s
your url is: https://yummy-hotels-jam.loca.lt
^C
